# Deep Learning Project : Automating Hyperparameter Tuning
# A Code Demo to the Optuna Framework for Future Data Scientists
# HOUSSENALY Ali

This code demo illustrates the use of the Optuna framework for automating hyperparameter tuning in machine learning and deep learning models.

It follows the presentation slides (DL_Project_Optuna_Presentation.pdf) and video provided alongside this notebook.

We will cover the following key aspects:
1. Code demo of Optuna for hyperparameter optimization in a machine learning context

2. Code demo of Optuna for hyperparameter optimization in a deep learning context

3. Extension: Using Optuna on the SDD mini-hackathon dataset to showcase its practical application

For the first 2 parts, we will use the Fashion MNIST dataset, which we have used in numerous classes.

For the last part, we will use the SDD mini-hackathon dataset, which is a more complex and realistic dataset.

## 0. Setup and Imports

In [8]:
import optuna
from optuna.trial import TrialState
from optuna.integration import CatBoostPruningCallback
from optuna.exceptions import ExperimentalWarning
import warnings
warnings.filterwarnings("ignore", category=ExperimentalWarning)


from optuna.visualization import plot_contour
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms


from torchvision import datasets, transforms
import numpy as np
import torch

from catboost import CatBoostClassifier

import pandas as pd

import os

In [9]:
def print_optuna_results(study):
    """Print a summary of the study results."""
    pruned_trials = study.get_trials(deepcopy=False,
                                     states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False,
                                       states=[TrialState.COMPLETE])

    print("\n Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

def visualize_optuna_results(study):
    """Visualize the study results with various plots."""
    fig1 = plot_optimization_history(study)
    fig1.show()

    fig2 = plot_param_importances(study)
    fig2.show()

    fig3 = plot_slice(study)
    fig3.show()

## 1. Code demo of Optuna for hyperparameter optimization in a machine learning context

Here we will optimize both the choice of classifier (among SVC, RandomForest and GradientBoosting) and their hyperparameters for the Fashion-MNIST dataset.

We define the parameters to try with trial.suggest_categorical, trial.suggest_float, and trial.suggest_int inside the objective function.

Then we return the metric to optimize (accuracy in this case).

For the optimization, we can set the pruners and samplers as desired.

Finally, we create a study and optimize it for a number of trials.

(Takes around 5 minutes on my computer)

In [10]:
# -----------------------------
# Load MNIST as numpy arrays
# -----------------------------
transform = transforms.Compose([transforms.ToTensor()])

mnist_train = datasets.MNIST(root=".", train=True, download=True, transform=transform)
mnist_test  = datasets.MNIST(root=".", train=False, download=True, transform=transform)

X_train = mnist_train.data.numpy().reshape(len(mnist_train), -1)
y_train = mnist_train.targets.numpy()

X_test = mnist_test.data.numpy().reshape(len(mnist_test), -1)
y_test = mnist_test.targets.numpy()

# Reduce dataset size for faster demo
X_train = X_train[:500]
y_train = y_train[:500]

X_test = X_test[:100]
y_test = y_test[:100]

# -----------------------------
# Optuna objective
# -----------------------------
def objective(trial):

    model_name = trial.suggest_categorical(
        "model",
        ["SVC", "RandomForest", "GradientBoosting"]
    )

    # -----------------------------
    # SVC
    # -----------------------------
    if model_name == "SVC":
        C = trial.suggest_float("svc_C", 1e-2, 1, log=True) # We try C between 1e-2 and 1
        kernel = trial.suggest_categorical("kernel", ["linear", "rbf"]) # We try linear and rbf kernels
        gamma = trial.suggest_float("gamma", 1e-2, 1, log=True) # We try gamma between 1e-2 and 1
        clf = sklearn.svm.SVC(C=C, kernel=kernel, gamma=gamma)

    # -----------------------------
    # Random Forest
    # -----------------------------
    elif model_name == "RandomForest":
        max_depth = trial.suggest_int("max_depth", 5, 30) # We try max_depth between 5 and 30
        n_estimators = trial.suggest_int("n_estimators", 50, 200) # We try n_estimators between 50 and 200
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10) # We try min_samples_split between 2 and 10
        clf = sklearn.ensemble.RandomForestClassifier(
            max_depth=max_depth,
            n_estimators=n_estimators,
            min_samples_split=min_samples_split
        )

    # -----------------------------
    # Gradient Boosting
    # -----------------------------
    else:
        lr = trial.suggest_float("learning_rate", 0.01, 0.5, log=True) # We try learning_rate between 0.01 and 0.5
        n_estimators = trial.suggest_int("gb_n_estimators", 50, 200) # We try n_estimators between 50 and 200
        max_depth = trial.suggest_int("gb_max_depth", 2, 10) # We try max_depth between 2 and 10
        clf = sklearn.ensemble.GradientBoostingClassifier(
            learning_rate=lr,
            n_estimators=n_estimators,
            max_depth=max_depth
        )

    # -----------------------------
    # Train + Evaluate
    # -----------------------------
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# -----------------------------
# Main: create study and optimize
# -----------------------------
if __name__ == "__main__":

    pruner = optuna.pruners.MedianPruner()
    sampler = optuna.samplers.TPESampler(seed=42)

    study = optuna.create_study(direction="maximize", pruner=pruner, sampler=sampler)
    study.optimize(objective, n_trials=30)

    print_optuna_results(study)
    visualize_optuna_results(study)

[I 2026-01-30 08:22:56,868] A new study created in memory with name: no-name-29fd6104-4dd8-4de3-9b1c-4a798e96e29f
[I 2026-01-30 08:22:57,104] Trial 0 finished with value: 0.82 and parameters: {'model': 'RandomForest', 'max_depth': 20, 'n_estimators': 73, 'min_samples_split': 3}. Best is trial 0 with value: 0.82.
[I 2026-01-30 08:22:57,183] Trial 1 finished with value: 0.82 and parameters: {'model': 'RandomForest', 'max_depth': 23, 'n_estimators': 53, 'min_samples_split': 10}. Best is trial 0 with value: 0.82.
[I 2026-01-30 08:22:57,237] Trial 2 finished with value: 0.14 and parameters: {'model': 'SVC', 'svc_C': 0.023270677083837805, 'kernel': 'rbf', 'gamma': 0.0730953983591291}. Best is trial 0 with value: 0.82.
[I 2026-01-30 08:22:57,395] Trial 3 finished with value: 0.84 and parameters: {'model': 'RandomForest', 'max_depth': 12, 'n_estimators': 105, 'min_samples_split': 6}. Best is trial 3 with value: 0.84.
[I 2026-01-30 08:22:57,459] Trial 4 finished with value: 0.14 and parameters:


 Study statistics: 
  Number of finished trials:  30
  Number of pruned trials:  0
  Number of complete trials:  30
Best trial:
  Value:  0.87
  Params: 
    model: RandomForest
    max_depth: 10
    n_estimators: 94
    min_samples_split: 4


We can see that the best model is SVC with specific hyperparameters.

## 2. Code demo of Optuna for hyperparameter optimization in a deep learning context

Here we will optimize a simple feedforward neural network for the Fashion-MNIST dataset.

We will optimize the following hyperparameters:
- Number of layers
- Number of units per layer
- Dropout ratio in each layer
- Optimizer type
- Learning rate

(Takes around 5 minutes on my computer)

In [11]:
DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 5) # We try between 1 and 5 layers
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128) # We try between 4 and 128 units
        layers.append(nn.Linear(in_features, out_features)) 
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5) # Dropout ratio between 0.2 and 0.5
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]) # We try Adam, RMSprop and SGD
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) # We try learning rate between 1e-5 and 1e-1
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print_optuna_results(study)
    visualize_optuna_results(study)

[I 2026-01-30 08:25:54,552] A new study created in memory with name: no-name-6985bc2a-c985-4299-ad60-93e187de5ff5


100%|██████████| 26.4M/26.4M [00:01<00:00, 13.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 925kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 10.3MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 16.6MB/s]
[I 2026-01-30 08:26:20,407] Trial 0 finished with value: 0.11875 and parameters: {'n_layers': 2, 'n_units_l0': 54, 'dropout_l0': 0.42761781787754993, 'n_units_l1': 45, 'dropout_l1': 0.39617593808549884, 'optimizer': 'SGD', 'lr': 4.5803004653535764e-05}. Best is trial 0 with value: 0.11875.
[I 2026-01-30 08:26:41,843] Trial 1 finished with value: 0.55 and parameters: {'n_layers': 3, 'n_units_l0': 10, 'dropout_l0': 0.3056690730472575, 'n_units_l1': 96, 'dropout_l1': 0.29930418291898303, 'n_units_l2': 90, 'dropout_l2': 0.45640021459337987, 'optimizer': 'Adam', 'lr': 0.00043263678261150644}. Best is trial 1 with value: 0.55.
[I 2026-01-30 08:27:05,619] Trial 2 finished with value: 0.6734375 and parameters: {'n_layers': 1, 'n_units_l0': 100, 'dropout_l0': 0.32503713725040717, 'optimi


 Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  70
  Number of complete trials:  30
Best trial:
  Value:  0.84453125
  Params: 
    n_layers: 1
    n_units_l0: 128
    dropout_l0: 0.2604274572894958
    optimizer: Adam
    lr: 0.017865584128930606


## 3. Extension: Using Optuna on the SDD mini-hackathon dataset to showcase its practical application

To extend this demo beyond classical uses of Optuna, we will apply it to the SDD mini-hackathon dataset.

We will use the CatBoost classifier and optimize its hyperparameters to improve performance on this specific dataset.

We will then be able to compare results with the leaderboard of the mini-hackathon and showcase the practical utility of Optuna.


In [12]:
df_train = pd.read_csv("mini-hackathon-data/train.csv")
df_test = pd.read_csv("mini-hackathon-data/test.csv")

X = df_train.drop('TARGET', axis=1)
y = df_train['TARGET']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(y.value_counts())

X shape: (225000, 324)
y shape: (225000,)
TARGET
False    204861
True      20139
Name: count, dtype: int64


In [13]:
X_train, X_val, y_train, y_val = train_test_split(
    X, 
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(X_train.shape, X_val.shape)

(180000, 324) (45000, 324)


In [14]:
def objective(trial):

    params = {
        "depth": trial.suggest_int("depth", 6, 12), # tree depth
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.2, log=True), # learning rate
        "iterations": trial.suggest_int("iterations", 1000, 6000), # number of trees
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 30, log=True), # L2 regularization
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.01, 2, log=True), # bagging temperature
        "subsample": trial.suggest_float("subsample", 0.5, 1.0), # subsample ratio
        "rsm": trial.suggest_float("rsm", 0.5, 1.0), # random subspace method ratio
        "border_count": trial.suggest_int("border_count", 32, 255),  # binning granularity
        "sampling_frequency": trial.suggest_categorical("sampling_frequency", ["PerTree", "PerTreeLevel"]), # sampling frequency
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 10),  # for categorical features
        "class_weights": [1, trial.suggest_int("pos_weight", 5, 20)], # class weights
        "loss_function": "Logloss", # loss function
        "eval_metric": "F1", # evaluation metric
        "random_state": 42, # for reproducibility
        "verbose": False # silent mode
    }

    model = CatBoostClassifier(**params)

    # Pruning callback cuts bad trials early
    pruning_callback = CatBoostPruningCallback(trial, "F1")

    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        early_stopping_rounds=60,
        verbose=False,
        callbacks=[pruning_callback]
    )

    # Get predicted probabilities
    probs = model.predict_proba(X_val)[:, 1]

    # Sweep thresholds from 0.1 to 0.9
    thresholds = np.linspace(0.1, 0.9, 81)
    f1s = [f1_score(y_val, probs > t) for t in thresholds]

    # Use the best F1 score
    score = max(f1s)

    # Save the best threshold for this trial
    best_threshold = thresholds[np.argmax(f1s)]
    trial.set_user_attr("best_threshold", best_threshold)

    return score


(Takes around 5 minutes on my computer)

In [15]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, show_progress_bar=True)

print_optuna_results(study)
visualize_optuna_results(study)

[I 2026-01-30 08:30:45,440] A new study created in memory with name: no-name-07779990-0bb0-47a5-9031-d6034b46668f


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2026-01-30 08:31:04,716] Trial 0 finished with value: 0.42152466367713004 and parameters: {'depth': 6, 'learning_rate': 0.04507823698643037, 'iterations': 3939, 'l2_leaf_reg': 4.59311597760305, 'bagging_temperature': 0.04329885054505414, 'subsample': 0.6429002286114547, 'rsm': 0.9552323070905852, 'border_count': 87, 'sampling_frequency': 'PerTreeLevel', 'one_hot_max_size': 5, 'pos_weight': 20}. Best is trial 0 with value: 0.42152466367713004.
[I 2026-01-30 08:31:16,048] Trial 1 finished with value: 0.41750588235294117 and parameters: {'depth': 6, 'learning_rate': 0.05200179646680585, 'iterations': 2671, 'l2_leaf_reg': 11.848172736800095, 'bagging_temperature': 1.8992361955623829, 'subsample': 0.7154068975680881, 'rsm': 0.8723666025734188, 'border_count': 83, 'sampling_frequency': 'PerTreeLevel', 'one_hot_max_size': 4, 'pos_weight': 8}. Best is trial 0 with value: 0.42152466367713004.
[I 2026-01-30 08:31:24,354] Trial 2 finished with value: 0.415648367681926 and parameters: {'depth':

The best score we obtain is around : 0.4240.

The best score in the leaderboard of the hackathon is : 0.43218.

We can clearly see here that with minimal code but with a highly tuned complex ML model, we reach similar performances to the best performances of the mini-hackathon.

This showcases the power of Optuna and hopefully, you will leverage this power for our future hackathon ! ;)